# Лабораторна робота №5

1. Спершу імпортуємо і підготуємо дані

In [63]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

Спершу імпортуємо дані і рандомно перемішаємо їх так як зараз вони надто структуровані, що може погіршити результати.

In [64]:
train_df = pd.read_csv('pml-training.csv', index_col=False).drop(columns=['Unnamed: 0'])
test_df = pd.read_csv('pml-testing.csv', index_col=False).drop(columns=['Unnamed: 0'])

train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)



Видалимо ті колонки в яких більше 80% пропущених значень, так як вони можуть негативно вплинути на результати.

In [65]:
threshold = 0.8
valid_columns = train_df.columns[train_df.isnull().mean() < threshold]
train_df = train_df[valid_columns]
test_df = test_df[train_df.columns.drop('classe')]




Видалимо нерелевантні колонки пов'язані з часом так як вони не будуть корисними для прогнозування на інших наборах даних.

In [66]:
cols_to_drop = ['raw_timestamp_part_1', 'raw_timestamp_part_2', 'cvtd_timestamp', 'new_window', 'num_window']
test_cols_to_drop = cols_to_drop + ['problem_id']
train_df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
test_df.drop(columns=test_cols_to_drop, inplace=True, errors='ignore')


Використаємо `One-hot encoding` для кодування категоріальної змінної `user_name`. А також розділимо дані на тренувальні дані та мітки

In [67]:

train_df = pd.get_dummies(train_df, columns=['user_name'], drop_first=True)
test_df = pd.get_dummies(test_df, columns=['user_name'], drop_first=True)


X = train_df.drop(columns=['classe'])
y = train_df['classe']

2. Тепер щоб вирішити поставлену задачу використаємо `RandomForestClassifier` та `10-fold cross-validation` для оцінки результатів моделі.

In [68]:
model = RandomForestClassifier(n_estimators=100, random_state=42)


cv_scores = cross_val_score(model, X, y, cv=10)
print(f'Cross-validation scores: {cv_scores}')
print(f'Mean cross-validation score: {cv_scores.mean()}')


model.fit(X, y)


y_pred = model.predict(test_df)
print("Predictions for the 20 test cases:\n", y_pred)


Cross-validation scores: [0.9979623  0.99745288 0.99643221 0.99745158 0.99592253 0.99643221
 0.99592253 0.99745158 0.99592253 0.99541284]
Mean cross-validation score: 0.9966363193078059
Predictions for the 20 test cases:
 ['B' 'A' 'B' 'A' 'A' 'E' 'D' 'B' 'A' 'A' 'B' 'C' 'B' 'A' 'E' 'E' 'A' 'B'
 'B' 'B']


Як бачимо, точність ориманих результатів є досить високою (>99% під час всіх десяти фолдів), що вказує на високу продуктивність моделі. Це в тому числі пояснюється вибором методу `Random Forest`, що є стійким до перенавчання завдяки його принципу роботи: побудові ансамблю дерев, усередненню їхніх передбачень і випадковому вибору ознак, що зменшує ризик запам’ятовування шуму та перенавчання.